# PAD-US v4

U.S. Geological Survey (USGS) Gap Analysis Project (GAP), 2024, Protected Areas Database of the United States (PAD-US) 4.0: U.S. Geological Survey data release, https://doi.org/10.5066/P96WBCHS. 




In [1]:
# pip install git+https://github.com/boettiger-lab/cng-python.git


  Cloning https://github.com/boettiger-lab/cng-python.git to /tmp/pip-req-build-bqjf1yv8
  Running command git clone --filter=blob:none --quiet https://github.com/boettiger-lab/cng-python.git /tmp/pip-req-build-bqjf1yv8
  Resolved https://github.com/boettiger-lab/cng-python.git to commit eeb63624d3559fa07f76cf26e4935a1f936bc72c
  Installing build dependencies ... -

In [2]:
import ibis
from ibis import _
con = ibis.duckdb.connect(extensions = ["spatial"])

# s3-write permissions
from cng.utils import set_secrets # 
import streamlit as st
set_secrets(con, st.secrets["MINIO_KEY"], st.secrets["MINIO_SECRET"])



In [5]:
from pathlib import Path
zip = "PADUS4_0_Geodatabase.zip"

if Path(zip).exists():
    import zipfile
    with zipfile.ZipFile(zip, 'r') as zip_ref:
        zip_ref.extractall()


In [7]:
#con.sql(f"select * from st_read_meta('{gdb}')").execute()  # no metadata?
# using duckdb + try_cast doesn't work either.
gdb = "PADUS4_0_Geodatabase.gdb"
layer = "PADUS4_0Combined_Proclamation_Marine_Fee_Designation_Easement"

fgb = "pad-us-4.fgb"

if Path(gdb).exists():
    ## UGH, duckdb still complains 'Geometry type 11 not supported'
    import geopandas
    gdf = geopandas.read_file(pad_us_4_gdb,
                layer = layer,
                driver = "pygrio")
    gdf.to_file()
    crs = gdf.crs.to_string()

In [6]:
# Sadly cannot do it this way
# con.read_geo(gdb, layer = layer, keep_wkb = True).mutate(geom = SHAPE.try_cast("geometry")).to_parquet('test.parquet')


In [8]:
# Geometry type 11 not supported
# SELECT TRY_CAST(wkb_geometry AS GEOMETRY) FROM st_read('some_dataset.fgb', keep_wkb := true);

# use fgb from geopandas instead

if Path(fgb).exists():
    (
        con
        .read_geo("pad-us-4.fgb")
        .mutate(geom = _.geom.convert('ESRI:102039', 'EPSG:4326'))
        .filter((_.FeatClass.isin(["Easement", "Fee"])) | (
            (_.FeatClass == "Proclamation") & (_.Mang_Name == "TRIB"))
            )
        .to_parquet('s3://public-biodiversity/pad-us-4/pad-us-4.parquet')
    )


In [12]:
# And here we go
"https://data.source.coop/cboettig/social-vulnerability/2022/SVI2022_US_tract.parquet"
svi = "s3://public-data/social-vulnerability/2022/SVI2022_US_tract.parquet" # faster with local

t1 = con.read_parquet(svi, "svi").select(_.ST_ABBR, _.STATE, _.FIPS, _.RPL_THEMES,  _.Shape, _.Shape_Area).rename(geom = "Shape")
t2 = con.read_parquet('s3://public-biodiversity/pad-us-4/pad-us-4.parquet').select(_.Unit_Nm, _.geom)

In [ ]:
(t1
 .left_join(t2, t1.geom.intersects(t2.geom))
 .to_parquet("s3://public-biodiversity/pad-us-4/pad-by-tract.parquet")
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [24]:
(t1
 .right_join(t2, t1.geom.intersects(t2.geom))
 .to_parquet("s3://public-biodiversity/pad-us-4/tract-by-pad.parquet")
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

CPU times: user 18h 32min 3s, sys: 21.7 s, total: 18h 32min 24s
Wall time: 9h 16min 32s


In [28]:
t1.count().execute(), t2.count().execute()

(84120, 414767)

In [9]:
# use ST number to get congressional districts
# https://www2.census.gov/geo/tiger/TIGER2024/CD/tl_2024_56_cd119.zip